In [38]:
import chromadb

chroma_client = chromadb.Client()

In [62]:
collection_name = "java-code-raptor-2"
try:
    collection = chroma_client.get_collection(name=collection_name)
except Exception:
    collection = chroma_client.create_collection(name=collection_name)

In [63]:
import json

raptor_data_path = "./java-result.json"
with open(raptor_data_path, "r") as f:
    raptor_data = json.load(f)

db_data = []
for index, data in raptor_data.items():
    print(f"Processing {index}...")
    for summary in data[0]:
        db_data.append({
            "document": summary["text"],
            "embedding": summary["embd"],
            "metadata": {
                "index": index,
                "cluster": summary["cluster"][0],
            }
        })

print(f"Total documents to insert: {len(db_data)}")

Processing 1...
Processing 2...
Total documents to insert: 16


In [64]:
collection.add(
    documents=[doc['document'] for doc in db_data],
    embeddings=[doc['embedding'] for doc in db_data],
    metadatas=[doc['metadata'] for doc in db_data],
    ids=[f"doc_{i}" for i, _ in enumerate(db_data)]
)

In [42]:
import ollama

def embed_text(text):
    response = ollama.embeddings(model="nomic-embed-text", prompt=text)
    return response['embedding']

In [ ]:
query = "sql schema"
query_embedding = embed_text(query)
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=10,
)
print("Query Results:")
context = ""
for result in results['documents'][0]:
    context += f"{result}\n"

print(context)

Query Results:
(File: README.md) This code is a basic API project using Java 11 + Maven + Spring Boot + PostgrSQL + Flyway. It includes instructions on how to set up the environment, install dependencies, configure the database connection, and create a simple project structure.

The code also provides useful commands for starting the server, installing dependencies, and skipping tests. Additionally, it explains the organization of the project, including the use of controllers, services, DAOs, repositories, and entities.
The provided documents are part of a Java-based Spring Boot project, including a basic HTML template, Maven configuration files, a README file with setup instructions, and a pom.xml file for dependency management. The files collectively outline the project's structure and provide necessary configurations for setting up the environment and running the application.
(File: pom.xml) This is a Maven project file, which is used to manage dependencies and build configurations 

In [44]:
import dspy
lm = dspy.LM("ollama_chat/llama3.2", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=lm)

In [45]:
rag = dspy.ChainOfThought("context, question -> response")

In [78]:
response = rag(context=context, question=query)
print("Response:")
print(response['response'])
print("Reasoning:")
print(response['reasoning'])

Response:
{ 
  "/users",
  "/users/{id}",
  "/users/new",
  "/users/{id}/update",
  "/users/{id}/delete"
}
Reasoning:
The provided Java Spring Boot project includes several URL paths that can be identified from the controller classes. These include:

*   `/users`: This path is used to fetch all users.
*   `/users/{id}`: This path is used to find one user by their ID.
*   `/users/new`: This path is used to add a new user.
*   `/users/{id}/update`: This path is used to update an existing user's details.
*   `/users/{id}/delete`: This path is used to delete a user by ID.
